# Output Schema

## Comprehensive Strategy

### No think label

In [24]:
# Most models can use the schema below.
no_think_schema = {
    "name": "no_think_label",
    "schema": {
        "type": "object",
        "description": "post-operative complications that arose during the medical case.",
        "properties": {
            "complications": {
                "type": "array",
                "description": "List of post-operative complications that arose during the medical case.",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "Name of the complication."
                        },
                        "grading": {
                            "type": "string",
                            "description": "Severity grading of the complication.",
                            "enum": [
                                "轻",
                                "中",
                                "重",
                                "Null"
                            ]
                        }
                    },
                    "required": [
                        "name",
                        "grading"
                    ]
                }
            }
        }
    }
}

# claude use tool_use schema like below
no_think_tools = [
    {
        "name": "medical_case_analysis",
        "description": "Analyze post-operative complications of medical case and provide structured analysis",
        "input_schema": {
            "type": "object",
            "description": "post-operative complications that arose during the medical case.",
            "properties": {
                "complications": {
                    "type": "array",
                    "description": "List of post-operative complications that arose during the medical case.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Name of the complication."
                            },
                            "grading": {
                                "type": "string",
                                "description": "Severity grading of the complication.",
                                "enum": [
                                    "轻",
                                    "中",
                                    "重",
                                    "Null"
                                ]
                            }
                        },
                        "required": [
                            "name",
                            "grading"
                        ]
                    }
                }
            },
            "required": [
                "complications"
            ]
        }
    }
]

### Having think label

In [22]:
# Most models can use the schema below.
think_label_schema = {
    "name": "having_think_label",
    "schema": {
        "type": "object",
        "properties": {
            "think": {
                "type": "array",
                "description": "A sequence of thoughts or considerations regarding the medical case.",
                "items": {
                    "type": "string"
                }
            },
            "complications": {
                "type": "array",
                "description": "List of post-operative complications that arose during the medical case.",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "Name of the complication."
                        },
                        "grading": {
                            "type": "string",
                            "description": "Severity grading of the complication.",
                            "enum": [
                                "轻",
                                "中",
                                "重",
                                "Null"
                            ]
                        }
                    },
                    "required": [
                        "name",
                        "grading"
                    ]
                }
            }
        },
        "required": [
            "think",
            "complications"
        ]
    }
}

# Claude tool_use schema (new format)
think_label_tools = [
    {
        "name": "medical_case_analysis",
        "description": "Analyze medical case complications and provide structured analysis",
        "input_schema": {
            "type": "object",
            "properties": {
                "think": {
                    "type": "array",
                    "description": "A sequence of thoughts or considerations regarding the medical case.",
                    "items": {
                        "type": "string"
                    }
                },
                "complications": {
                    "type": "array",
                    "description": "List of post-operative complications that arose during the medical case.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Name of the complication."
                            },
                            "grading": {
                                "type": "string",
                                "description": "Severity grading of the complication.",
                                "enum": [
                                    "轻",
                                    "中",
                                    "重",
                                    "Null"
                                ]
                            }
                        },
                        "required": [
                            "name",
                            "grading"
                        ]
                    }
                }
            },
            "required": [
                "think",
                "complications"
            ]
        }
    }
]

## Targeted Strategy

In [ ]:
# Most models can use the schema below.
targeted_schema = {
    "name": "targeted_schema",
    "schema": {
        "type": "object",
        "properties": {
            "think": {
                "type": "string",
                "description": "Thoughts or considerations regarding the complication.",
            },
            "bool": {
                "type": "boolean",
                "description": "Whether the complication is diagnosed."
            },
            "grading": {
                "type": "string",
                "description": "Severity grading of the complication.",
                "enum": [
                    "轻",
                    "中",
                    "重",
                    "Null"
                ]
            }
        },
        "required": [
            "think",
            "bool",
            "grading"
        ]
    }
}

# Define request funciton

## Openai compatible api

In [6]:
# Includes GPT series, DeepSeek series, Grok series, Qwen series, etc.
import asyncio
from openai import AsyncOpenAI

api_key = "<your_api_key>"
api_base = "<your_api_base>"

client = AsyncOpenAI(api_key=api_key, base_url=api_base)

timeout = 2000
sleep_time = 5
# response_format = "no_think_schema" # can be "no_think_schema", "think_label_schema", "targeted_schema"

async def fetch_openai_response(prompt, model:str, client=client, response_format=None):
    while True:
        try:
            completion_args = {
                "model": model,
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.6,
                "top_p": 0.95,
                "max_tokens": 2000,
                "timeout": timeout
            }           
            if response_format: # some model may not support json schema, this parameter should be modified according to the model
                completion_args["response_format"] = {
                    "type": "json_schema",
                    "json_schema": response_format
                }
            print("开始请求...")
            response = await client.chat.completions.create(**completion_args)
            if response and response.choices:
                print("获取结果成功")
                # Retrieve reasoning content and final response
                if "openrouter" in api_base:
                    reasoning_content = getattr(response.choices[0].message, 'reasoning', "") # 获取思维链
                else:
                    reasoning_content = getattr(response.choices[0].message, 'reasoning_content', "") # 获取思维链
                content = response.choices[0].message.content
                # return a dictionary containing reasoning content and final response
                if content:
                    return {
                        "reasoning_content": reasoning_content,
                        "content": content
                    }
                else:
                    continue
            else:
                print("No response received from OpenAI, retrying...")
                await asyncio.sleep(sleep_time)
        except asyncio.TimeoutError:
            print(f"Request timed out after {timeout} seconds, retrying...")
            await asyncio.sleep(sleep_time)
        except Exception as e:
            print(f"Error: {e}, retrying...")
            await asyncio.sleep(sleep_time)


## Gemini

In [ ]:
from google import genai
from google.genai import types
import asyncio
import time

api_key = "YOUR_GOOGLE_API_KEY"

async def fetch_gemini_response(prompt, model_name, response_format):    
    while True:
        try:
            client = genai.Client(
                api_key=api_key,
            )

            contents = [
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_text(text=prompt), 
                   ],
                ),
            ]
            if response_format:
                config = {
                    'response_mime_type': 'application/json',
                    'response_schema': response_format["schema"]
                }
            else:
                config = {}

            response = await asyncio.to_thread(
                client.models.generate_content,
                model=model_name,
                contents=contents,
                config=config
            )
            
            return response.text
        except Exception as e:
            print(f"Google API error: {e}, retrying...")
            time.sleep(2)

## Claude

In [19]:
# Claude API request function
import anthropic
import time

api_key = "YOUR_ANTHROPIC_API_KEY_HERE"

def fetch_claude_response(prompt, model, think=True, tool_use=None):
    while True:
        try:
            claude_client = anthropic.Anthropic(api_key=api_key)
            completion_args = {
                "model": model,
                "max_tokens": 2000,
                "system": [
                    {"type": "text", "text": prompt["system"], "cache_control": {"type": "ephemeral"}}, # use cache to reduce token usage
                ],
                "messages": [
                    {"role": "user", "content": prompt["user"]}
                ]
            }
            if think:
                completion_args["thinking"] = {
                    "type": "enabled",
                    "budget_tokens": 2000,
                }
            if tool_use:
                completion_args["tools"] = tool_use
                
            response = claude_client.messages.create(**completion_args)

            if not response:
                raise ValueError("No valid response received from Claude API.")

            print("Successfully retrieved Claude result")
            print(f"Cache usage: {response.usage}")
            
            # Process response content
            has_thinking = any(
                block.type == "thinking" for block in response.content
            )
            if has_thinking:
                print("thinking")
                reasoning_content = "\n".join(block.thinking for block in response.content if block.type == "thinking")
                print(reasoning_content)
            else:
                reasoning_content = ""
            
            # process tool use input
            tool_input = None
            # find tool_use block in content
            for block in response.content:
                if block.type == "tool_use":
                    tool_input = block.input
                    break

            if tool_input:
                content = tool_input
            elif hasattr(response, 'content') and response.content:
                # backup plan: if no tool_use but has content
                content = response.content[0].text if hasattr(response.content[0], 'text') else "{}"
            else:
                content = "{}"
            
            return {
                "reasoning_content": reasoning_content,
                "content": content
            }
        except Exception as e:
            print(f"Claude API error: {e}, retrying...")
            time.sleep(2)

# Request response

In [4]:
# read prompts

import json
with open("prompt/comprehensive_prompts_nothink.json", "r") as f:
    comprehensive_prompts_nothink = json.load(f)

with open("prompt/comprehensive_prompts.json", "r") as f:
    comprehensive_prompts = json.load(f)

with open("prompt/comprehensive_prompts_modified.json", "r") as f:
    comprehensive_prompts_modified = json.load(f)

print(len(comprehensive_prompts_nothink))
print(len(comprehensive_prompts))
print(len(comprehensive_prompts_modified))

1
1
1


### Openai and Gemini

In [6]:
final_dict = {}

In [16]:
# This section initiates requests to the large language model.
# We will manually define the calling function, model, output directory, and other parameters for each request.
# Below are three illustrative examples.
# First is openai compatible models (gpt series, deepseek series, etc.)
output = "results/raw/o1_comprehensive_nothink.json"
model = "openai/o1" # can be other openai compatible models
prompt = comprehensive_prompts_nothink # can be comprehensive_prompts, comprehensive_prompts_modified

response_format = no_think_schema # can be think_label_schema, targeted_schema.

start = 0 # start from the first prompt
end = len(comprehensive_prompts_nothink) # end at the last prompt
batch_size = 5 # enable batch processing
sleep_time = 5 # sleep time between requests

# batch processing
for i in range(start, end, batch_size):
    batch_prompts = list(comprehensive_prompts_nothink.values())[i:i + batch_size]
    batch_pt_nos = list(comprehensive_prompts_nothink.keys())[i:i + batch_size]
    batch_tasks = [fetch_openai_response(prompt, model, response_format) for prompt in batch_prompts]
    
    batch_responses = await asyncio.gather(*batch_tasks)
    
    if "final_dict" not in globals():
        final_dict = {}
    
    for pt_no, response in zip(batch_pt_nos, batch_responses):
        final_dict[pt_no] = response
        print(f"Patient {pt_no} retrieved successfully, {i+1}/{end} requests completed")
    
    await asyncio.sleep(sleep_time)
    
    with open(output, "w", encoding="utf-8") as f:
        json.dump(final_dict, f, indent=4, ensure_ascii=False)


开始请求...
获取结果成功
Patient 1 获取成功，已完成1/1个请求


In [ ]:
final_dict = {}

In [18]:
# Second is Gemini series models
output = "results/raw/gemini2.5_flash_comprehensive_nothink.json"
model = "gemini-2.5-flash" # can be gemini pro
prompt = comprehensive_prompts_nothink # can be comprehensive_prompts, comprehensive_prompts_modified

response_format = no_think_schema # can be think_label_schema, targeted_schema.

start = 0 # start from the first prompt
end = len(comprehensive_prompts_nothink) # end at the last prompt
batch_size = 5 # enable batch processing
sleep_time = 5 # sleep time between requests

# batch processing
for i in range(start, end, batch_size):
    batch_prompts = list(comprehensive_prompts_nothink.values())[i:i + batch_size]
    batch_pt_nos = list(comprehensive_prompts_nothink.keys())[i:i + batch_size]
    batch_tasks = [fetch_gemini_response(prompt, model, response_format) for prompt in batch_prompts] # can be fetch_gemini_response
    
    batch_responses = await asyncio.gather(*batch_tasks)
    
    if "final_dict" not in globals():
        final_dict = {}
    
    for pt_no, response in zip(batch_pt_nos, batch_responses):
        final_dict[pt_no] = response
        print(f"Patient {pt_no} retrieved successfully, {i+1}/{end} requests completed")
    
    await asyncio.sleep(sleep_time)
    
    with open(output, "w", encoding="utf-8") as f:
        json.dump(final_dict, f, indent=4, ensure_ascii=False)

Patient 1 获取成功，已完成1/1个请求


## Claude

In [ ]:
final_dict = {}

In [26]:
# We need to separate the prompts to leverage caching and reduce token consumption.
output = "results/raw/claude3_7_sonnet_comprehensive_nothink.json"
model = "claude-3-7-sonnet-20250219"
prompt = comprehensive_prompts_nothink

for i, (k, v) in enumerate(prompt.items()):
    pt_no = k
    prompt_combined = v.split("### 病历资料")
    response = fetch_claude_response({"system": prompt_combined[0], "user": "### 病历资料\n" + prompt_combined[1]}, model=model, think=False, tool_use=no_think_tools)
    final_dict[pt_no] = response
    print(f"Processing {i+1}/{len(comprehensive_prompts_nothink)}")
    with open(output, 'w', encoding='utf-8') as json_file:
        json.dump(final_dict, json_file, ensure_ascii=False, indent=4)

Successfully retrieved Claude result
Cache usage: Usage(cache_creation_input_tokens=3628, cache_read_input_tokens=0, input_tokens=8222, output_tokens=319, service_tier='standard')
Processing 1/1


## Targeted requests

In [1]:
final_dict = {}

In [3]:
import json

with open("prompt/targeted_prompts.json", "r") as f:
    prompts = json.load(f)

In [ ]:
import json
import asyncio
from openai import AsyncOpenAI
import aiohttp

epoches = 1
batch_size = 1
start = 0
end = len(prompts)

response_format = None # if deepseek, use None because it doesn't support response_format
# Reset API base and key for DeepSeek
api_base = "https://api.deepseek.com/v1"
api_key = "YOUR_DEEPSEEK_API_KEY_HERE"
output = "results/raw/deepseek_r1_targeted.json"
model = "deepseek-reasoner" # can be other openai compatible models

client = AsyncOpenAI(api_key=api_key, base_url=api_base)

async def single_pt_response(prompts:dict):
    result_dict = {}
    for k, v in prompts.items():
        print(f"Processing {k}...")
        response = await fetch_openai_response(v, model = model, client=client, response_format=response_format)
        result_dict[k] = response
    return result_dict

async with aiohttp.ClientSession() as session:

    for epoch in range(1, epoches+1):
        print(f"Starting to process {epoch}/{epoches}")
        final_dict = {}
        # Process requests in batches
        for i in range(start, end, batch_size):
            batch_prompts = list(prompts.values())[i:i + batch_size]
            batch_pt_nos = list(prompts.keys())[i:i + batch_size]
            
            # Create tasks for current batch
            batch_tasks = [single_pt_response(prompt) for prompt in batch_prompts]
            print(f"Now processing records {i} to {i+batch_size}")
            
            # Wait for current batch to complete
            batch_responses = await asyncio.gather(*batch_tasks)
            print("Processing completed")
           
            # Save results
            for pt_no, response in zip(batch_pt_nos, batch_responses):
                final_dict[pt_no] = response
            
            # Pause after each batch to avoid too frequent requests
            await asyncio.sleep(1)
    
        # Save all results to file after each batch
        with open(output, 'w', encoding='utf-8') as json_file:
            json.dump(final_dict, json_file, ensure_ascii=False, indent=4)

Starting to process 1/1
Now processing records 0 to 1
开始请求急性肾损伤...
开始请求...
获取结果成功
开始请求急性呼吸窘迫综合征 (ARDS)...
开始请求...
获取结果成功
开始请求吻合口破裂...
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求心律失常...
开始请求...
获取结果成功
开始请求心脏骤停...
开始请求...
获取结果成功
开始请求心源性肺水肿...
开始请求...
获取结果成功
开始请求深静脉血栓...
开始请求...
获取结果成功
开始请求谵妄...
开始请求...
获取结果成功
开始请求胃肠道出血...
开始请求...
获取结果成功
开始请求感染（来源不明）...
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求血流感染（实验室确诊）...
开始请求...
获取结果成功
开始请求心肌梗死...
开始请求...
获取结果成功
开始请求非心脏手术后心肌损伤...
开始请求...
获取结果成功
开始请求肺炎...
开始请求...
获取结果成功
开始请求麻痹性肠梗阻...
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求术后出血...
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求...
获取结果成功
开始请求肺栓塞...
开始请求...
获取结果成功
开始请求中风...
开始请求...
获取结果成功
开始请求浅表手术部位感染...
开始请求...
获取结果成功
开始请求深部手术部位感染...
开始请求...
获取结果成功
开始请求器官/腔隙手术部位感染...
开始请求...
获取结果成功
开始请求泌尿道感染...
开始请求...
获取结果成功
Processing completed
